# Question 1: 

Introspection would not do a good job of informing AI efforts of modelling human cognitive behavior for a number of reasons, the first of which is bias. I believe that it is impossible for a person to examine their own psyche and mental process in an unbiased way. I find it hard to imagine that anyone has truly neutral feelings toward themselves, whether it be liking or disliking, and any non-neutral emotion toward a person introduces bias into their analysis. Furthermore, many mental health professionals will not engage in treatment or examining of anyone they have a personal history with because it is difficult for them to eliminate their bias because of that history. No one has more history with any person other than themself! Because of this unavoidable bias, I don't think any person could examine their own thought process and come out of the situation with an accurate, unbiased account of how it works. It simply doesn't seem possible. 


A secondary reason that introspection is a bad system to base AI off of is that people are different and they think differently, so even if a person's thought could be correctly analyzed and represented in AI, there's no guarantee that that depiction of a human brain would think and process in a way that is applicable to everyone. There would need to be a different AI for every person, which is an impossible feat. There is simply no way to model the human psyche digitally, especially when powered by introspection.

# Question 2: 

State: current path of cities, including its length

Action: Optional actions include all unvisited cities that could be visited next. The result of the action is the city added on to the path.

In [20]:
"""
This module implements local search on the Travelling Salesman Problem
Adapted from abs.py by kvlinden for lab02, cs344

@author: ksn7
@version: 19feb2019
"""

from search import Problem, hill_climbing, simulated_annealing, \
    exp_schedule, genetic_search
from random import randrange
import math


class TSP(Problem):
    """
    State: current path of cities with the lengths between them
    Move: list of cities that could still be visited
    """
    
    # Need to save ordered path list and distances between cities
    def __init__(self, initial, distances):
        self.initial = initial
        self.distances = distances
        
    # Each action tells you which city to swap with the city sequentially after it. 
    # Ex) action 2 would switch cities 2 and 3 in the list
    # First and last cities cannot be swapped with any others
    def actions(self, state):
        choices = []
        for i in range(1, len(state)-1):
            for j in range(1, len(state)-1):
                if i != j:
                    choices.append((i,j))
        return choices
    
    # result receives a state and a choice of which first element to swap
    # returns list with chosen element and its successor swapped
    def result(self, state, choice):
        new_state = state[:]
        swap1 = new_state[choice[0]]
        swap2 = new_state[choice[1]]
        new_state[choice[0]] = swap2
        new_state[choice[1]] = swap1
        return new_state
    
    # Check pair of cities in both orders to find dictionary entry, then return path cost plus added distance
    def value(self, state):
        distance = 0
        for i in range(0, len(state)-1):
            c1 = state[i]
            c2 = state[i+1]
            if (c1, c2) in self.distances:
                distance += self.distances[(c1, c2)]
            else:
                distance += self.distances[(c2, c1)]
        negated_dist = distance * -1
        return negated_dist


if __name__ == '__main__':
    
    # Given list of cities
    cities = ['Atlanta', 'Baltimore', 'Cadillac', 'Denver', 'Elmhurst', 'Franklin', 'Georgetown', 'Helena']
    
    # copy over given list of cities, appending first to end to make a loop
    initial = cities[:]
    initial.append(cities[0])
    
    # Create an empty dictionary, then fill it with pair of all cities and random distances between them
    distances = {}
    for i in range(0, len(cities)):
        for j in range(0, len(cities)):
            c1 = cities[i]
            c2 = cities[j]
            if c1 != c2:
                if (c2, c1) not in distances:
                    distances[(c1, c2)] = randrange(1, 10, 1)
    print('Distance dictionary: ')
    print(distances)
    print()
    
    p = TSP(initial, distances)

    # Solve the problem using hill-climbing.
    hill_solution = hill_climbing(p)
    print('Hill-climbing solution       Order: ' + str(hill_solution)
          + '\tDistance: ' + str(-1 * p.value(hill_solution))
          )

    # Solve the problem using simulated annealing.
    annealing_solution = simulated_annealing(
        p,
        exp_schedule(k=20, lam=0.005, limit=1000)
    )
    print('Simulated annealing solution Order: ' + str(annealing_solution)
          + '\tDistance: ' + str(-1 * p.value(annealing_solution))
          )


Distance dictionary: 
{('Atlanta', 'Baltimore'): 9, ('Atlanta', 'Cadillac'): 7, ('Atlanta', 'Denver'): 8, ('Atlanta', 'Elmhurst'): 6, ('Atlanta', 'Franklin'): 9, ('Atlanta', 'Georgetown'): 6, ('Atlanta', 'Helena'): 4, ('Baltimore', 'Cadillac'): 2, ('Baltimore', 'Denver'): 4, ('Baltimore', 'Elmhurst'): 7, ('Baltimore', 'Franklin'): 5, ('Baltimore', 'Georgetown'): 9, ('Baltimore', 'Helena'): 6, ('Cadillac', 'Denver'): 6, ('Cadillac', 'Elmhurst'): 3, ('Cadillac', 'Franklin'): 9, ('Cadillac', 'Georgetown'): 4, ('Cadillac', 'Helena'): 6, ('Denver', 'Elmhurst'): 9, ('Denver', 'Franklin'): 1, ('Denver', 'Georgetown'): 4, ('Denver', 'Helena'): 2, ('Elmhurst', 'Franklin'): 9, ('Elmhurst', 'Georgetown'): 3, ('Elmhurst', 'Helena'): 2, ('Franklin', 'Georgetown'): 6, ('Franklin', 'Helena'): 6, ('Georgetown', 'Helena'): 7}

Hill-climbing solution       Order: ['Atlanta', 'Baltimore', 'Cadillac', 'Elmhurst', 'Georgetown', 'Franklin', 'Denver', 'Helena', 'Atlanta']	Distance: 30
Simulated annealing sol

Simulated annealing works better because hill-climbing is guaranteed to find a local minimum. However, since this problem can be so complicated, you are more likely to find an optimal solution if you sometimes take worse choices, shaking up the order of the cities to find a path that is signficantly shorter. The hill-climbing solution makes fewer changes and is less likely to find an optimal solution.

# Question 3:

In [12]:
""""
This module implements course scheduling in the form of a CSP
Adapted from the Zebra function in csp.py from CS344 lab03

@author Kelsey Brouwer
@version 20feb2019
"""

from csp import CSP, parse_neighbors, min_conflicts

def Schedule():
    
    """Return an instance of the Course Scheduling Puzzle."""
    # Class info includes professor, class times, and classrooms
    profs = 'Adams Bailey Plantinga Schuurman'.split()
    times = 'mwf900 mwf1030 mwf1230 mwf230 tth830'.split()
    rooms = 'nh253 sb382'.split()
    classes = '108 112 212 214 384 394'.split()
    
    
    # Create all options for professor, time, and room. These will be our domains
    domains = {}
    options = []
    for i in range(0, len(profs)):
        for j in range(0, len(times)):
            for k in range(0, len(rooms)):
                options.append((profs[i], times[j], rooms[k]))
                
    for i in range(0, len(classes)):
        domains[classes[i]] = options

    # Neighbors tells which variables interact and need to be checked in constraints
    neighbors = parse_neighbors('108: 112 212 214 384 394; 112: 212 214 384 394; 212: 214 384 394; 214: 384 394; 384: 394', classes)

    # Function to check the course assignments meet the constraints
    def class_constraint(A, a, B, b, recurse=0):
        
        # Pull out all important course information
        prof1 = a[0]
        time1 = a[1]
        room1 = a[2]
        prof2 = b[0]
        time2 = b[1]
        room2 = b[2]       
        
        if (A == '108') and (prof1 != 'Schuurman'):
            return False
        if (A == '112') and (prof1 != 'Adams'):
            return False
        if (A == '212') and (prof1 != 'Plantinga'):
            return False
        if (A == '214') and (prof1 != 'Adams'):
            return False
        if (A == '384') and (prof1 != 'Schuurman'):
            return False
        if (A == '394') and (prof1 != 'Bailey'):
            return False
        if (A != B) and (prof1 == prof2) and (time1 == time2):
            return False
        if (A != B) and (time1 == time2) and (room1 == room2):
            return False     
        if recurse == 0:
            return class_constraint(B, b, A, a, 1)
        return True
    return CSP(classes, domains, neighbors, class_constraint)

puzzle = Schedule()
solution = min_conflicts(puzzle)
print(solution)

{'108': ('Schuurman', 'tth830', 'sb382'), '112': ('Adams', 'mwf900', 'nh253'), '212': ('Plantinga', 'mwf1030', 'nh253'), '214': ('Adams', 'tth830', 'nh253'), '384': ('Schuurman', 'mwf1230', 'nh253'), '394': ('Bailey', 'mwf230', 'sb382')}


##### 